In [2]:
import pandas as pd
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [194]:
class IsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        self.data_activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        self.data_spins_correlation = self.data_activations_matrix.corr().values
        lind = np.tril_indices_from(self.data_spins_correlation)
        self.data_spins_correlation[lind]=0
        self.data_spins_average = self.data_activations_matrix.mean().values
        
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]

        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.data_spins_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        self.H = self.data_activations_matrix.sum()/self.num_samples_data
        # Try with random H
        #self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 500:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            corr_model_exp = np.zeros(self.J.shape)
            mean_model_exp = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                
                # Calculate the expectation of the correlations
                for i in range(0, self.num_spins):
                    for j in range(i+1, self.num_spins):
                        corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_spins_correlation - corr_model_exp)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(self.J - stepJ).sum().sum()
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH + deltaJ
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(self.H)
        print(self.data_spins_correlation)
        print(self.J)
        
    def random_walk(self, max_steps=40):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        randomWalkLength = 40
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(randomWalkLength)   
            model_activations_matrix = pd.DataFrame(model_activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
            # Calculate teh data expectations
            model_spins_correlation = model_activations_matrix.corr().values
            lind = np.tril_indices_from(model_spins_correlation)
            model_spins_correlation[lind]=0
            model_spins_average = model_activations_matrix.mean().values
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_spins_correlation - model_spins_correlation)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(self.J - stepJ).sum().sum()
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - model_spins_average)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH + deltaJ
            
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch +=1

        print(self.H)
        print(self.data_spins_correlation)
        print(self.J)
        

In [195]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]

In [196]:
ising = IsingModel(activations_list, lr=0.0005)

In [198]:
ising.train_metropolis(max_epochs = 600)


Epoch 1 TotalParamVariation 1.0386251871067014
Epoch 2 TotalParamVariation 1.0409685621272495
Epoch 3 TotalParamVariation 1.0411801991156893
Epoch 4 TotalParamVariation 1.0421684409112768
Epoch 5 TotalParamVariation 1.042256708046085
Epoch 6 TotalParamVariation 1.0433404201006085
Epoch 7 TotalParamVariation 1.0424107375902232
Epoch 8 TotalParamVariation 1.0443897299082576
Epoch 9 TotalParamVariation 1.0434225142315678
Epoch 10 TotalParamVariation 1.0435380811498343
Epoch 11 TotalParamVariation 1.0443926240436652
Epoch 12 TotalParamVariation 1.0450715537406852
Epoch 13 TotalParamVariation 1.0452122539945772
Epoch 14 TotalParamVariation 1.0452558066241338
Epoch 15 TotalParamVariation 1.0464874071014318
Epoch 16 TotalParamVariation 1.0467225530182034
Epoch 17 TotalParamVariation 1.0480430468213973
Epoch 18 TotalParamVariation 1.048737313495888
Epoch 19 TotalParamVariation 1.0487808553572215
Epoch 20 TotalParamVariation 1.0510161251091168
Epoch 21 TotalParamVariation 1.0505111317133733
Epo

Epoch 239 TotalParamVariation 1.2609566281079856
Epoch 240 TotalParamVariation 1.2617781232535044
Epoch 241 TotalParamVariation 1.2627687579847833
Epoch 242 TotalParamVariation 1.2642370099350346
Epoch 243 TotalParamVariation 1.265117877159387
Epoch 244 TotalParamVariation 1.2650064889705646
Epoch 245 TotalParamVariation 1.2654940761698699
Epoch 246 TotalParamVariation 1.2670801262687994
Epoch 247 TotalParamVariation 1.2678627547271266
Epoch 248 TotalParamVariation 1.2692237941500748
Epoch 249 TotalParamVariation 1.2715848214802778
Epoch 250 TotalParamVariation 1.2719943954350843
Epoch 251 TotalParamVariation 1.2727102655466938
Epoch 252 TotalParamVariation 1.2730257973612211
Epoch 253 TotalParamVariation 1.2727395553309466
Epoch 254 TotalParamVariation 1.2744108827803853
Epoch 255 TotalParamVariation 1.2744993700907488
Epoch 256 TotalParamVariation 1.276992412141643
Epoch 257 TotalParamVariation 1.2778919962418893
Epoch 258 TotalParamVariation 1.278121629337511
Epoch 259 TotalParamVar

Epoch 479 TotalParamVariation 1.47171665969698
Epoch 480 TotalParamVariation 1.4732120463220644
Epoch 481 TotalParamVariation 1.473936845196124
Epoch 482 TotalParamVariation 1.474072106356728
Epoch 483 TotalParamVariation 1.4759441487935128
Epoch 484 TotalParamVariation 1.4771685333089528
Epoch 485 TotalParamVariation 1.478127205699743
Epoch 486 TotalParamVariation 1.4803198004186575
Epoch 487 TotalParamVariation 1.4808650568171797
Epoch 488 TotalParamVariation 1.4812364690962814
Epoch 489 TotalParamVariation 1.4814306407479463
Epoch 490 TotalParamVariation 1.4827289445150402
Epoch 491 TotalParamVariation 1.4838973253169272
Epoch 492 TotalParamVariation 1.4850542461734635
Epoch 493 TotalParamVariation 1.4844580796393076
Epoch 494 TotalParamVariation 1.4861909087591767
Epoch 495 TotalParamVariation 1.486343221897879
Epoch 496 TotalParamVariation 1.487244814579109
Epoch 497 TotalParamVariation 1.4886216211604526
Epoch 498 TotalParamVariation 1.4881576504814569
Epoch 499 TotalParamVariati

In [193]:
ising.train(max_epochs= 600)

Epoch 1 TotalParamVariation 1.039811912152953
Epoch 2 TotalParamVariation 1.0399087674052108
Epoch 3 TotalParamVariation 1.040006570628456
Epoch 4 TotalParamVariation 1.040104252054163
Epoch 5 TotalParamVariation 1.0402018118244487
Epoch 6 TotalParamVariation 1.0402992500813
Epoch 7 TotalParamVariation 1.0403965669665733
Epoch 8 TotalParamVariation 1.0404937626219963
Epoch 9 TotalParamVariation 1.0405908371891675
Epoch 10 TotalParamVariation 1.0406877908095546
Epoch 11 TotalParamVariation 1.0407846236244978
Epoch 12 TotalParamVariation 1.0408813357752067
Epoch 13 TotalParamVariation 1.0409779274027626
Epoch 14 TotalParamVariation 1.0410743986481172
Epoch 15 TotalParamVariation 1.0411707496520932
Epoch 16 TotalParamVariation 1.041266980555385
Epoch 17 TotalParamVariation 1.041363091498557
Epoch 18 TotalParamVariation 1.041459082622046
Epoch 19 TotalParamVariation 1.0415549540661595
Epoch 20 TotalParamVariation 1.041650705971076
Epoch 21 TotalParamVariation 1.0417463384768457
Epoch 22 To

Epoch 350 TotalParamVariation 1.1902994002023455
Epoch 351 TotalParamVariation 1.1907099683912197
Epoch 352 TotalParamVariation 1.1911202965127805
Epoch 353 TotalParamVariation 1.1915303847833418
Epoch 354 TotalParamVariation 1.1919402334190152
Epoch 355 TotalParamVariation 1.1923498426357122
Epoch 356 TotalParamVariation 1.1927592126491422
Epoch 357 TotalParamVariation 1.1931683436748148
Epoch 358 TotalParamVariation 1.1935772359280383
Epoch 359 TotalParamVariation 1.1939858896239213
Epoch 360 TotalParamVariation 1.1943943049773713
Epoch 361 TotalParamVariation 1.194802482203096
Epoch 362 TotalParamVariation 1.1952104215156034
Epoch 363 TotalParamVariation 1.1956181231292011
Epoch 364 TotalParamVariation 1.1960255872579972
Epoch 365 TotalParamVariation 1.196432814115901
Epoch 366 TotalParamVariation 1.1968398039166215
Epoch 367 TotalParamVariation 1.1972465568736685
Epoch 368 TotalParamVariation 1.1976530732003536
Epoch 369 TotalParamVariation 1.198059353109789
Epoch 370 TotalParamVar

#### Only H

In [93]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        self.data_activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        self.data_spins_correlation = self.data_activations_matrix.corr().values
        lind = np.tril_indices_from(self.data_spins_correlation)
        self.data_spins_correlation[lind]=0
        self.data_spins_average = self.data_activations_matrix.mean().values
        
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = self.data_activations_matrix.sum()/self.num_samples_data
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            mean_model_exp = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]

            oldH = copy.deepcopy(self.H)

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = np.sum(np.absolute(stepH))
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH
            
            diffH = np.sum(np.absolute(self.H - oldH))

            
            
            
            print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 3))
            print(mean_model_exp, 'Average Model')
            print(self.data_spins_average, 'Average Data')
            print(self.data_spins_average - mean_model_exp, 'Diff')
            #diffH = ['%.8f' % n for n in diffH]
            print(round(diffH, 15), 'DiffH')
            print()
            
            epoch += 1

        print(self.H)
        #print(self.data_spins_correlation)
        
    def random_walk(self, max_steps=40):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        randomWalkLength = 40
        
        print('Epoch 0')
        print(self.H)
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(randomWalkLength)   
            model_activations_matrix = pd.DataFrame(model_activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
            # Calculate teh data expectations
            model_spins_correlation = model_activations_matrix.corr().values
            lind = np.tril_indices_from(model_spins_correlation)
            model_spins_correlation[lind]=0
            model_spins_average = model_activations_matrix.mean().values
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - model_spins_average)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            totalParamVariation = abs(self.H - oldH).sum
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            print(model_spins_average, 'Average Model')
            print(self.data_spins_average, 'Average Data')
            print(self.data_spins_average - model_spins_average, 'Diff')
            print()

            
            epoch +=1

        #print(self.H)
        #print(self.data_spins_correlation)
        
        print(self.data_spins_average, 'Spins average')


In [94]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = SimpleIsingModel(activations_list, lr=0.0005)


In [95]:
s_ising.train( max_epochs = 10000)

Epoch 1 TotalParamVariation 0.001
[-0.0034047  -0.00461958 -0.01843688 -0.01528672] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2534047  -0.24538042  0.51843688  0.01528672] Diff
0.000516254361829 DiffH

Epoch 2 TotalParamVariation 0.001
[-0.003278   -0.00474226 -0.01817775 -0.01527908] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.253278   -0.24525774  0.51817775  0.01527908] Diff
0.000515996281032 DiffH

Epoch 3 TotalParamVariation 0.001
[-0.00315136 -0.00486489 -0.01791874 -0.01527144] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.25315136 -0.24513511  0.51791874  0.01527144] Diff
0.000515738328051 DiffH

Epoch 4 TotalParamVariation 0.001
[-0.00302479 -0.00498745 -0.01765986 -0.01526381] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.25302479 -0.24501255  0.51765986  0.01526381] Diff
0.000515480502843 DiffH

Epoch 5 TotalParamVariation 0.001
[-0.00289828 -0.00510996 -0.01740111 -0.01525618] Average Model
[ 0.25 -0.25  0.5   0.  ] Average 

Epoch 119 TotalParamVariation 0.0
[ 0.01111673 -0.01867953  0.01127079 -0.01441087] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23888327 -0.23132047  0.48872921  0.01441087] Diff
0.000486671908515 DiffH

Epoch 120 TotalParamVariation 0.0
[ 0.01123616 -0.01879515  0.01151513 -0.01440367] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23876384 -0.23120485  0.48848487  0.01440367] Diff
0.000486428616932 DiffH

Epoch 121 TotalParamVariation 0.0
[ 0.01135552 -0.01891071  0.01175934 -0.01439646] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23864448 -0.23108929  0.48824066  0.01439646] Diff
0.00048618544807 DiffH

Epoch 122 TotalParamVariation 0.0
[ 0.01147483 -0.01902621  0.01200342 -0.01438927] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23852517 -0.23097379  0.48799658  0.01438927] Diff
0.000485942401884 DiffH

Epoch 123 TotalParamVariation 0.0
[ 0.01159408 -0.01914166  0.01224739 -0.01438208] Average Model
[ 0.25 -0.25  0.5   0.  ] Average D

[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22385147 -0.21676917  0.45798855  0.01350391] Diff
0.00045605654924 DiffH

Epoch 250 TotalParamVariation 0.0
[ 0.02626038 -0.03333909  0.04224004 -0.01349716] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22373962 -0.21666091  0.45775996  0.01349716] Diff
0.000455828823233 DiffH

Epoch 251 TotalParamVariation 0.0
[ 0.02637217 -0.0334473   0.04246851 -0.01349041] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22362783 -0.2165527   0.45753149  0.01349041] Diff
0.000455601213866 DiffH

Epoch 252 TotalParamVariation 0.0
[ 0.0264839  -0.03355546  0.04269686 -0.01348367] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2235161  -0.21644454  0.45730314  0.01348367] Diff
0.000455373721091 DiffH

Epoch 253 TotalParamVariation 0.0
[ 0.02659558 -0.03366356  0.04292509 -0.01347693] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22340442 -0.21633644  0.45707491  0.01347693] Diff
0.00045514634486 DiffH

Epoch 254 Total

Epoch 373 TotalParamVariation 0.0
[ 0.03959473 -0.04624532  0.06946599 -0.01269203] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21040527 -0.20375468  0.43053401  0.01269203] Diff
0.000428692991166 DiffH

Epoch 374 TotalParamVariation 0.0
[ 0.03969977 -0.04634698  0.06968022 -0.01268569] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21030023 -0.20365302  0.43031978  0.01268569] Diff
0.000428479358036 DiffH

Epoch 375 TotalParamVariation 0.0
[ 0.03980476 -0.04644859  0.06989433 -0.01267934] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21019524 -0.20355141  0.43010567  0.01267934] Diff
0.000428265835495 DiffH

Epoch 376 TotalParamVariation 0.0
[ 0.03990969 -0.04655014  0.07010833 -0.012673  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21009031 -0.20344986  0.42989167  0.012673  ] Diff
0.000428052423495 DiffH

Epoch 377 TotalParamVariation 0.0
[ 0.04001456 -0.04665165  0.07032221 -0.01266667] Average Model
[ 0.25 -0.25  0.5   0.  ] Average 

[ 0.19827319 -0.19201329  0.40582185  0.01195881] Diff
0.000404033568716 DiffH

Epoch 493 TotalParamVariation 0.0
[ 0.05182569 -0.0580824   0.09437925 -0.01195283] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19817431 -0.1919176   0.40562075  0.01195283] Diff
0.000403832748701 DiffH

Epoch 494 TotalParamVariation 0.0
[ 0.05192451 -0.05817803  0.09458025 -0.01194686] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19807549 -0.19182197  0.40541975  0.01194686] Diff
0.000403632033391 DiffH

Epoch 495 TotalParamVariation 0.0
[ 0.05202328 -0.05827362  0.09478115 -0.01194089] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19797672 -0.19172638  0.40521885  0.01194089] Diff
0.000403431422736 DiffH

Epoch 496 TotalParamVariation 0.0
[ 0.052122   -0.05836915  0.09498193 -0.01193492] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.197878   -0.19163085  0.40501807  0.01193492] Diff
0.000403230916687 DiffH

Epoch 497 TotalParamVariation 0.0
[ 0.05222067 -0.05

...

Epoch 9492 TotalParamVariation 0.0
[ 2.47310170e-01 -2.47390212e-01  4.90259959e-01 -1.32710472e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00268983 -0.00260979  0.00974004  0.00013271] Diff
7.586184677e-06 DiffH

Epoch 9493 TotalParamVariation 0.0
[ 2.47311433e-01 -2.47391437e-01  4.90263658e-01 -1.32644117e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00268857 -0.00260856  0.00973634  0.00013264] Diff
7.583057916e-06 DiffH

Epoch 9494 TotalParamVariation 0.0
[ 2.47312695e-01 -2.47392662e-01  4.90267356e-01 -1.32577795e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00268731 -0.00260734  0.00973264  0.00013258] Diff
7.579932468e-06 DiffH

Epoch 9495 TotalParamVariation 0.0
[ 2.47313956e-01 -2.47393886e-01  4.90271053e-01 -1.32511506e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00268604 -0.00260611  0.00972895  0.00013251] Diff
7.576808332e-06 DiffH

Epoch 9496 TotalParamVariation 0.0
[ 2.47315217e-01 -2.47395109e-01  4.9

Epoch 9611 TotalParamVariation 0.0
[ 2.47456334e-01 -2.47532021e-01  4.90690359e-01 -1.25042662e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00254367 -0.00246798  0.00930964  0.00012504] Diff
7.223163901e-06 DiffH

Epoch 9612 TotalParamVariation 0.0
[ 2.47457528e-01 -2.47533180e-01  4.90693893e-01 -1.24980141e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00254247 -0.00246682  0.00930611  0.00012498] Diff
7.220189455e-06 DiffH

Epoch 9613 TotalParamVariation 0.0
[ 2.47458722e-01 -2.47534338e-01  4.90697426e-01 -1.24917651e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00254128 -0.00246566  0.00930257  0.00012492] Diff
7.217216256e-06 DiffH

Epoch 9614 TotalParamVariation 0.0
[ 2.47459915e-01 -2.47535495e-01  4.90700957e-01 -1.24855192e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00254009 -0.00246451  0.00929904  0.00012486] Diff
7.214244304e-06 DiffH

Epoch 9615 TotalParamVariation 0.0
[ 2.47461107e-01 -2.47536652e-01  4.9

Epoch 9732 TotalParamVariation 0.0
[ 2.47596804e-01 -2.47668306e-01  4.91108273e-01 -1.17700098e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0024032  -0.00233169  0.00889173  0.0001177 ] Diff
6.872158971e-06 DiffH

Epoch 9733 TotalParamVariation 0.0
[ 2.47597932e-01 -2.47669400e-01  4.91111646e-01 -1.17641248e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00240207 -0.0023306   0.00888835  0.00011764] Diff
6.869331626e-06 DiffH

Epoch 9734 TotalParamVariation 0.0
[ 2.47599059e-01 -2.47670494e-01  4.91115019e-01 -1.17582427e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00240094 -0.00232951  0.00888498  0.00011758] Diff
6.866505466e-06 DiffH

Epoch 9735 TotalParamVariation 0.0
[ 2.47600186e-01 -2.47671587e-01  4.91118390e-01 -1.17523636e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00239981 -0.00232841  0.00888161  0.00011752] Diff
6.863680489e-06 DiffH

Epoch 9736 TotalParamVariation 0.0
[ 2.47601312e-01 -2.47672680e-01  4.9

[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00226943 -0.00220191  0.00848956  0.00011073] Diff
6.535815667e-06 DiffH

Epoch 9855 TotalParamVariation 0.0
[ 2.47731637e-01 -2.47799123e-01  4.91513660e-01 -1.10677932e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00226836 -0.00220088  0.00848634  0.00011068] Diff
6.533129116e-06 DiffH

Epoch 9856 TotalParamVariation 0.0
[ 2.47732702e-01 -2.47800156e-01  4.91516878e-01 -1.10622593e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0022673  -0.00219984  0.00848312  0.00011062] Diff
6.530443688e-06 DiffH

Epoch 9857 TotalParamVariation 0.0
[ 2.47733766e-01 -2.47801188e-01  4.91520095e-01 -1.10567281e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00226623 -0.00219881  0.00847991  0.00011057] Diff
6.527759384e-06 DiffH

Epoch 9858 TotalParamVariation 0.0
[ 2.47734829e-01 -2.47802220e-01  4.91523310e-01 -1.10511998e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00226517 -0.00219778  0.00847

Epoch 9975 TotalParamVariation 0.0
[ 2.47855880e-01 -2.47919665e-01  4.91891154e-01 -1.04230987e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00214412 -0.00208033  0.00810885  0.00010423] Diff
6.218766022e-06 DiffH

Epoch 9976 TotalParamVariation 0.0
[ 2.47856886e-01 -2.47920641e-01  4.91894227e-01 -1.04178871e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00214311 -0.00207936  0.00810577  0.00010418] Diff
6.216212036e-06 DiffH

Epoch 9977 TotalParamVariation 0.0
[ 2.47857892e-01 -2.47921617e-01  4.91897300e-01 -1.04126782e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00214211 -0.00207838  0.0081027   0.00010413] Diff
6.213659118e-06 DiffH

Epoch 9978 TotalParamVariation 0.0
[ 2.47858897e-01 -2.47922593e-01  4.91900371e-01 -1.04074718e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0021411  -0.00207741  0.00809963  0.00010407] Diff
6.211107267e-06 DiffH

Epoch 9979 TotalParamVariation 0.0
[ 2.47859902e-01 -2.47923567e-01  4.9